In [2]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

In [4]:
import get_data

In [5]:
(x_train, x_test), (y_train, y_test) = get_data.get_all_splits()
print(x_train, x_test, y_train, y_test, sep="\n")

elephant
squirrel
sheep
cow
butterfly
[array([ 55,  59,  36, ..., 222, 222, 134])
 array([174, 212, 235, ..., 215, 190, 149])
 array([252, 252, 252, ..., 253, 253, 253]) ... 0 0 0]
[array([253, 253, 253, ..., 253, 253, 253])
 array([250, 250, 250, ..., 227, 228, 246])
 array([163, 193, 193, ..., 113, 121,  48]) ... 0 0 0]
[array([ 55,  59,  36, ..., 222, 222, 134])
 array([174, 212, 235, ..., 215, 190, 149])
 array([252, 252, 252, ..., 253, 253, 253]) ... 0 0 0]
[array([253, 253, 253, ..., 253, 253, 253])
 array([250, 250, 250, ..., 227, 228, 246])
 array([163, 193, 193, ..., 113, 121,  48]) ... 0 0 0]


/Users/enerney/PycharmProjects/cs254final/get_data.py:57: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return (np.array(master_x_train), np.array(master_x_test)), (np.array(master_y_train), np.array(master_y_test))


In [6]:
def re_split(l):
    x = []
    y = []
    num_labels = 0
    total = 0
    for item in l:
        total += 1
        if item.shape == ():
            num_labels += 1
            y.append(item)
        else:
            x.append(item)
        
    print(num_labels, total, sep="/")
    return x, y
        

In [7]:
x_test1 = []
y_test1 = []
x_train1 = []
y_train1 = []
ind = 0
for l in [x_train, x_test, y_train, y_test]:
    l1, l2 = re_split(l)
    for i in range(len(l1)):
        # gets rid of one image with the wrong dimensions
        if len(l1[i]) == 12288:
            if ind % 2 == 0:
                x_train1.append(l1[i])
                y_train1.append(l2[i])
            else: 
                x_test1.append(l1[i])
                y_test1.append(l2[i])
        
        
    ind += 1
        


6145/12290
2051/4102
6145/12290
2051/4102


In [8]:
y_train = np.array(y_train1)
y_test = np.array(y_test1)

In [9]:
x_train = np.array(x_train1)
x_test = np.array(x_test1)

In [10]:
# after more work than I intended, the arrays are in the correct shapes
# this is to normalize the data (pixel values between 0 and 1)
x_train = x_train / 255
x_test = x_test / 255

In [12]:
model = tf.keras.Sequential([
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(5)
])

model.add(tf.keras.layers.Conv2D(64, (3, 3), activation='relu', input_shape=(64, 64, 3)))

model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [13]:
model.fit(x_train, y_train, epochs=10)

Epoch 1/10


ValueError: in user code:

    /Users/enerney/anaconda3/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:805 train_function  *
        return step_function(self, iterator)
    /Users/enerney/anaconda3/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:795 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    /Users/enerney/anaconda3/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.py:1259 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    /Users/enerney/anaconda3/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.py:2730 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    /Users/enerney/anaconda3/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.py:3417 _call_for_each_replica
        return fn(*args, **kwargs)
    /Users/enerney/anaconda3/lib/python3.8/site-packages/t

In [67]:
est_loss, test_acc = model.evaluate(x_test,  y_test, verbose=2)

print('\nTest accuracy:', test_acc)

128/128 - 0s - loss: 4.1557 - accuracy: 0.4314

Test accuracy: 0.43143975734710693
